## 포트홀 탐지 모델

yolov5를 이용하여 도로의 포트홀(파인 구멍)을 인식하는 모델을 학습시키고, 검증하고, 테스트 해 보는 실습을 진행해 봅시다!

본 프로젝트는 리눅스 환경 or 구글 코랩 환경(https://colab.research.google.com/) 에서 진행하는 것을 가정합니다! 


### 데이터셋 다운로드 (아래의 설명대로 따라해보세요!)

우선 탐지 모델을 학습시키고 검증하며, 테스트할 데이터셋을 가져와 봅시다!

* 포트홀 데이터셋: https://public.roboflow.com/object-detection/pothole <br>
위 사이트 들어가기 -> '665 Images' 클릭 -> Format을 YOLO v5 PyTorch로 설정 -> Continue -> Raw URL 클릭하면 뜨는 링크 복사


1.   665 Images 클릭!
   
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbOtJAB%2FbtrJ1YdJ2Vj%2FNqzg54YeRqHD4mVPpW6DL0%2Fimg.png">

2. 포맷 설정 (이때 팝업이 뜨지 않는다면 페이지 로그인을 진행해야 한다!)
   
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fby2o0e%2FbtrJ3aEr1Lv%2F0dsgnedczHehCW6IDbeIK0%2Fimg.png">

3. URL 복사

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdTG8wr%2FbtrJ3Tvt2JA%2Fw8uasH2BKtdrZ0eIY2krmk%2Fimg.png">



***
이제 yolov5 레포지토리를 clone하고 필요한 종속성(패키지, 모듈)을 설치해봅시다!

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

import torch
from IPython.display import Image, clear_output

다음은 데이터셋을 다운받아 압축을 푸는 단계를 진행해 보겠습니다!

curl 명령어를 이용하여 위의 데이터셋을 zip 파일로 다운받은 뒤 압축을 푸는 방향으로 진행할 것입니다

In [ ]:
%mkdir pothole
%cd pothole
!curl -L "" > roboflow.zip #큰따옴표 안에 링크 붙여넣기
!unzip roboflow.zip -d ./
!rm roboflow.zip x
%cd ..

데이터셋을 보면  train, valid, test 폴더로 나눠져 있는 것을 확인할 수 있습니다!

* train 데이터셋 : 모델을 학습시키는데 사용되는 데이터셋
* valid 데이터셋 : 모델 학습 중간 중간에 모델이 얼마나 잘 학습하는지 확인할때 사용되는 중간 테스트(validation, 검증) 데이터셋
* test 데이터셋 : 모델 학습에 사용되지 않고 학습이 완료된 모델이 얼마나 잘 감지하는지 확인할 때 사용하는 데이터셋

이제, 나눠진 데이터셋의 이미지의 이름을 텍스트 파일에 모아 정리할 것입니다!

* glob.glob : 파라미터로 입력된 경로와 패턴에 해당되는 모든 파일의 파일명을 리스트 형식으로 가져옵니다.
  * `*` 키워드 : 임의의 길이의 모든 파일명을 가져온다
  * `?` 키워드 : 하나의 길이의 파일명을 가져온다
  * `**` 키워드 : 해당하는 폴더의 하위 폴더까지 모두 검색하여 가져온다



In [ ]:
from glob import glob

train_img_list = glob('pothole/train/images/*.jpg')
test_img_list = glob('pothole/test/images/*.jpg')
valid_img_list = glob('pothole/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list)) #465 67 133이 출력되어야 함!

In [ ]:
import yaml
with open('pothole/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')
    
with open('pothole/test.txt', 'w') as f:
    f.write('\n'.join(test_img_list) + '\n')

with open('pothole/valid.txt', 'w') as f:
    f.write('\n'.join(valid_img_list) + '\n')   

한 번, 잘 저장이 됐는지 test.txt의 내용을 확인해봅시다!

In [ ]:
%cat pothole/test.txt

모델이 학습/추론 때 데이터를 찾을 때 사용하는 `data.yaml` 파일 또한 현재 위치에 맞게 수정해야 합니다. 일일이 파일을 읽어 오지 말고 새로운 매직 명령어를 선언하여 사용해 봅시다!

`IPython.core.magic.register_line_cell_magic` 메서드는 함수를 커스텀 매직 명령어로 만드어 줍니다.

`register_line_cell_magic` 로 만든 매직 명령어는 `%%` 키워드를 이용하여 불러올 수 있습니다. 이제 직접 사용해 봅시다!

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%cat pothole/data.yaml

In [ ]:
%%writetemplate pothole/data.yaml

train: ./pothole/train/images
test: ./pothole/train/images
val: ./pothole/valid/images

nc: 1
names: ['pothole']

In [ ]:
%cat pothole/data.yaml

### 모델 구성

이전까지의 모델은 미리 만들어진 모델을 이용했다면, 이젠 직접 모델을 만들어 학습할 예정입니다.
 
yolov5의 모델 중 s 버전의 모델을 수정해 봅시다. s 버전의 모델은 `models/yolov5s.yaml`에 위치합니다.

In [ ]:
import yaml
with open("pothole/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
    
%cat models/yolov5s.yaml

In [ ]:
%%writetemplate models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat models/custom_yolov5s.yaml

모델 수정이 완료되었습니다! 

### 학습(Training)

자, 이제 학습을 위한 데이터 및 모델 준비를 완료했습니다! 이제 직접 학습을 시작해보도록 하겠습니다.

yolov5에서 학습은 `train.py` 가 담당합니다. `train.py`의 입력 파라미터는 다음과 같습니다.

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

이제, 학습을 시작하도록 하겠습니다.

In [ ]:
%%time
!python train.py --img 640 --batch 32 --epochs 100 --data ./pothole/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pothole_results --cache

### tensorboard로 학습 경과 보기

TensorBoard는 모델 학습, 추론 등 다양한 기능을 시각화하여 보여주는 툴입니다.

TensorBoard를 이용하여 모델 학습의 Loss, 학습률 등을 한 눈에 보기 쉽게 그래프로 시각화해보겠습니다

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls runs/train/pothole_results/

실제 학습의 결과 사진을 확인해보겠습니다

In [ ]:
Image(filename='runs/train/pothole_results/results.png', width=1000)

In [ ]:
Image(filename='runs/train/pothole_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='runs/train/pothole_results/val_batch0_labels.jpg', width=1000)

얼추 학습이 완료된 것 같습니다! 이제 검증을 해볼까요?

### 검증(Validation)

검증(validation)은 학습 도중 모델이 어떤 방향으로 학습을 해야 하는지를 알려주고 잘 학습되고 있는지 '검증'하는 단계입니다

학습 데이터 중 일부를 이용하여 실제 추론을 해보고 결과가 맞는지를 다양한 방법으로 유추하여 학습의 방향성을 잡아주는 단계를 이야기합니다!

이제 yolov5의 검증을 해보겠습니다! 검증은 `val.py` 파일로 진행합니다.

In [ ]:
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./pothole/data.yaml --img 640 --iou 0.65 --half

In [ ]:
!python val.py --weights runs/train/pothole_results/weights/best.pt --data ./pothole/data.yaml --img 640 --task test

### 추론(Inference)

이제 마지막으로 추론을 진행해서 결과를 확인해 봅시다!

추론은 이전 실습에서 확인한 바와 같이 `detect.py`로 진행합니다.

In [ ]:
%ls runs/train/pothole_results/weights

In [ ]:
!python detect.py --weights runs/train/pothole_results/weights/best.pt --img 640 --conf 0.4 --source ./pothole/test/images

In [ ]:
import glob
import random
from IPython.display import Image, display

image_name = random.choice(glob.glob('runs/detect/exp/*.jpg'))
display(Image(filename=image_name))

### 모델 내보내기

잘 학습한 모델은 반드시 다음에 사용할 일이 있을 것입니다.

그런데 학습한 모델을 가지고 있지 않으면 다시 학습과 검증을 거쳐야 하는데 정말 성가시고 비효율적인 일이지요

하지만 yolov5는 자동으로 가장 좋은 결과를 보여주는 버전의 모델을 `.pt` 파일로 자동으로 저장해줍니다

이 파일을 나의 구글 드라이브에 저장해보겠습니다.

아래의 셀을 실행하여 출력되는 링크에 들어가면 코드가 나오며, 이를 복사한 후 빈칸에 입력하세요
- 입력 시 자신의 google drive에 학습한 모델이 저장됩니다

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%mkdir /content/drive/My\ Drive/pothole
%cp runs/train/pothole_results/weights/best.pt /content/drive/My\ Drive/pothole

yolov5를 이용하여 데이터 준비, 학습, 검증, 추론 그리고 모델 저장까지 가볍게 진행해 보았습니다!

AI 세계는 무궁무진해서, 다양한 기법을 이용해서 학습 효율과 성능을 높이며 데이터 전처리를 진행하여 데이터의 양과 질을 늘이는 등 많은 방법과 모델이 있습니다!

이제 YOLO를 넘어 다양한 모델을 다양한 데이터와 함께 다뤄보도록 해봅시다!

* 캐글(https://www.kaggle.com/) : 다양한 데이터셋을 제공해주며 AI 관련 프로젝트 및 컴퍼티션을 개최합니다
* AI 허브(https://aihub.or.kr/) : 다양한 무료 빅데이터를 제공해주는 사이트
* 공공데이터포탈(https://www.data.go.kr/) : 정부에서 운영하는 국가 관련 공개 데이터 제공 사이트
